Params to change:

In [2]:
first_n_layers_to_probe = [3,4]


In [3]:
mlqa_eval_languages = ["mlqa.en.en", "mlqa.ar.en", "mlqa.en.ar", "mlqa.ar.ar", "mlqa.es.es", "mlqa.en.de", "mlqa.de.de"]

batch_size = 8

model_name = 'bert-base-multilingual-cased'
#model_name = 'xlm-roberta-base'
#model_name = 'jakobwes/xlm_roberta_squad_v1.1'
#model_name = 'salti/bert-base-multilingual-cased-finetuned-squad'

Install dev-version of Hugginface transformers from GitHub-repo, as well as Hugginface datasets:

In [4]:
!pip install datasets
!git clone https://github.com/huggingface/transformers
!cd transformers && pip install .

     |████████████████████████████████| 225kB 9.2MB/s 
     |████████████████████████████████| 245kB 18.5MB/s 
     |████████████████████████████████| 112kB 17.7MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 72664, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (381/381), done.
remote: Total 72664 (delta 400), reused 489 (delta 281), pack-reused 71962
Receiving objects: 100% (72664/72664), 56.19 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (51519/51519), done.
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 11.0MB/s 
     |████████████████████████████████| 3.3MB 32.4MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2259809 sha256=41ee4cd4e246d6b2d119fdaa02c75f5a6766f200eda4a340d287b4d33b94db8a
  Stored in directory: /tmp/pip

# Run Experiment

This code is based of [here](https://github.com/huggingface/transformers/tree/master/examples/pytorch/question-answering) -- the Hugginface QA-environment. We pull a modified version of `run_qa.py` from our GitHub-repo, but use the standard `trainer.py` and `utils_qa.py`.



In [5]:
!curl -L -O https://raw.githubusercontent.com/jakobwes/QA_layer_freeze/main/experiment_code/run_qa.py
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/question-answering/trainer_qa.py
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/question-answering/utils_qa.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28626  100 28626    0     0   125k      0 --:--:-- --:--:-- --:--:--  125k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4007  100  4007    0     0  21313      0 --:--:-- --:--:-- --:--:-- 21313
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22276  100 22276    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
import os

for layers in first_n_layers_to_probe:
  print("\n \n \n \n Running model "+model_name+" for layers 1 to "+ str(layers) + "\n \n \n \n")

  # Model path and folder name (without slash)
  model_saving_path = "./"+str(layers)+"_layer_model/"
  model_saving_folder_name = "./"+str(layers)+"_layer_model"

  # Get and save model
  model = AutoModelForQuestionAnswering.from_pretrained(model_name, num_hidden_layers=layers)
  model.save_pretrained(model_saving_path)

  # Get and save tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)  
  tokenizer.save_pretrained(model_saving_path)

  # Output dir path and output dir name (without slash) for squad-trained model
  model_output_dir_name = model_saving_folder_name+"_ft"
  model_output_dir_path = model_saving_folder_name+"_ft/"

  batch_size_str = str(batch_size)

  # Run training and save in output-dir
  !python run_qa.py  \
      --model_name_or_path $model_saving_folder_name \
      --output_dir $model_output_dir_name \
      --num_train_epochs 3.0 \
      --dataset_name squad \
      --do_train \
      --do_eval \
      --per_device_train_batch_size $batch_size_str \
      --per_device_eval_batch_size $batch_size_str \
      --max_seq_length 384 \
      --doc_stride 128


  for language_pair in mlqa_eval_languages:

    escaped_language_pair_str = language_pair.replace(".", "_")
    eval_output_dir = "./eval/"+str(layers)+"_layer_model/"+escaped_language_pair_str+"/"

    # Run evaluation with weights from output_dir_model and save in eval_output_dir
    !python run_qa.py  \
        --model_name_or_path $model_output_dir_path \
        --output_dir $eval_output_dir \
        --dataset_name mlqa \
        --dataset_config_name $language_pair \
        --do_eval \
        --do_predict \
        --per_device_train_batch_size $batch_size_str \
        --per_device_eval_batch_size $batch_size_str \
        --max_seq_length 384 \
        --doc_stride 128 


 
 
 
 Running model bert-base-multilingual-cased for layers 1 to 3
 
 
 



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.7.attention.self.key.weight', 'bert.encoder.layer.8.intermediate.dense.weight', 'bert.encoder.layer.9.attention.self.value.weight', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.6.attention.self.value.weight', 'cls.seq_relationship.weight', 'bert.encoder.layer.8.attention.self.query.weight', 'bert.encoder.layer.7.attention.self.query.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.6.output.dense.weight', 'bert.encoder.layer.6.attention.self.query.weight', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.key.weight', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.8.attention.self.value.


2021-05-16 14:24:07.371865: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/16/2021 14:24:09 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/16/2021 14:24:09 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./3_layer_model_ft, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May16_14-24-09_69feeac9c45a, logging_strategy=IntervalStrategy.STEPS, logging_first_st

# Get Results

Extract the result and save in a dict:

In [ ]:
import json

results = {}
results["mlqa"] = {}
results["squad"] = {}

# Extract mlqa-results
for layers in first_n_layers_to_probe:

  results_per_layer = {}
  for language_pair in mlqa_eval_languages:

    results_per_language_pair_and_layer = {}

    escaped_language_pair_str = language_pair.replace(".", "_")
    eval_output_dir = "./eval/"+str(layers)+"_layer_model/"+escaped_language_pair_str

    with open(eval_output_dir+"/all_results.json") as json_file:
      results_per_language_pair_and_layer["all results"] = json.load(json_file)

    with open(eval_output_dir+"/eval_results.json") as json_file:
      results_per_language_pair_and_layer["eval results"] = json.load(json_file)

    with open(eval_output_dir+"/predict_results.json") as json_file:
      results_per_language_pair_and_layer["predict results"] = json.load(json_file)
      
    results_per_layer[escaped_language_pair_str] = results_per_language_pair_and_layer

  results["mlqa"]["layers " + str(layers)] = results_per_layer

# Extract squad-results
for layers in first_n_layers_to_probe:

  results_per_layer = {}

  model_saving_folder_name = "./"+str(layers)+"_layer_model"
  model_output_dir_path = model_saving_folder_name+"_ft/"

  with open(model_output_dir_path+"/all_results.json") as json_file:
    results_per_layer["all results"] = json.load(json_file)

  with open(model_output_dir_path+"/eval_results.json") as json_file:
    results_per_layer["eval results"] = json.load(json_file)

  with open(model_output_dir_path+"/train_results.json") as json_file:
    results_per_layer["train results"] = json.load(json_file)
    
  results["squad"]["layers " + str(layers)] = results_per_layer




In [ ]:
results